## `Task` Do feature selection as per metods taught is session 54 on SECOM dataset.

Dataset Link : https://archive.ics.uci.edu/ml/datasets/SECOM

Drive Link : https://docs.google.com/spreadsheets/d/1dFCe1zgokabsiEr6BbWmMJtiMefkrChpJWLiG_0dDkk/edit?usp=share_link

In [69]:
# Write your Code here

### `Solution`

In [70]:
import pandas as pd
import numpy as np

data = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQtBXo5cBnDsM2fmfHPm6u72KGUS5FjPHNGMxOfYjA9-CAhmnRpwkIw_rOR3sANJIToiUU__6fbBvig/pub?gid=572763137&single=true&output=csv")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 592 entries, Time to Pass/Fail
dtypes: float64(590), int64(1), object(1)
memory usage: 7.1+ MB


In [71]:
data.head()

,Time,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,-0.0034,0.9455,202.4396,0.0,7.9558,414.8710,10.0433,0.9680,192.3963,12.5190,1.4026,-5419.00,2916.50,-4043.75,751.00,0.8955,1.7730,3.0490,64.2333,2.0222,0.1632,3.5191,83.3971,9.5126,50.6170,64.2588,49.3830,66.3141,86.9555,...,0.78,0.1827,5.7349,0.3363,39.8842,3.2687,1.0297,1.0344,0.4385,0.1039,42.3877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,533.8500,2.1113,8.95,0.3157,3.0624,0.1026,1.6765,14.9509,NaN,NaN,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,-0.0148,0.9627,200.5470,0.0,10.1548,414.7347,9.2599,0.9701,191.2872,12.4608,1.3825,-5441.50,2604.25,-3498.75,-1640.25,1.2973,2.0143,7.3900,68.4222,2.2667,0.2102,3.4171,84.9052,9.7997,50.6596,64.2828,49.3404,64.9193,87.5241,...,1.33,0.2829,7.1196,0.4989,53.1836,3.9139,1.7819,0.9634,0.1745,0.0375,18.1087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,535.0164,2.4335,5.92,0.2653,2.0111,0.0772,1.1065,10.9003,0.0096,0.0201,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,0.0013,0.9615,202.0179,0.0,9.5157,416.7075,9.3144,0.9674,192.7035,12.5404,1.4123,-5447.75,2701.75,-4047.00,-1916.50,1.3122,2.0295,7.5788,67.1333,2.3333,0.1734,3.5986,84.7569,8.6590,50.1530,64.1114,49.8470,65.8389,84.7327,...,0.85,0.0857,7.1619,0.3752,23.0713,3.9306,1.1386,1.5021,0.3718,0.1233,24.7524,267.064,0.9032,1.10,0.6219,0.4122,0.2562,0.4119,68.8489,535.0245,2.0293,11.21,0.1882,4.0923,0.0640,2.0952,9.2721,0.0584,0.0484,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,-0.0033,0.9629,201.8482,0.0,9.6052,422.2894,9.6924,0.9687,192.1557,12.4782,1.4011,-5468.25,2648.25,-4515.00,-1657.25,1.3137,2.0038,7.3145,62.9333,2.6444,0.2071,3.3813,84.9105,8.6789,50.5100,64.1125,49.4900,65.1951,86.6867,...,39.33,0.6812,56.9303,17.4781,161.4081,35.3198,54.2917,1.1613,0.7288,0.2710,62.7572,268.228,0.6511,7.32,0.1630,3.5611,0.0670,2.7290,25.0363,530.5682,2.0253,9.33,0.1738,2.8971,0.0525,1.7585,8.5831,0.0202,0.0149,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,-0.0072,0.9569,201.9424,0.0,10.5661,420.5925,10.3387,0.9735,191.6037,12.4735,1.3888,-5476.25,2635.25,-3987.50,117.00,1.2887,1.9912,7.2748,62.8333,3.1556,0.2696,3.2728,86.3269,8.7677,50.2480,64.1511,49.7520,66.1542,86.1468,...,1.98,0.4287,9.7608,0.8311,70.9706,4.9086,2.5014,0.9778,0.2156,0.0461,22.0500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,532.0155,2.0275,8.83,0.2224,3.1776,0.0706,1.6597,10.9698,NaN,NaN,NaN,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


In [72]:
# Logistic Model Before Feature Selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Dropping Time Column
data = data.drop('Time', axis=1)

# Filling NaN Values with Random
for column in data.columns:
    # Print("Column-", column)
    # Get the minimum and maximum values of the column
    min_value = data[column].min()
    max_value = data[column].max()
    
    # Generate Random numbers within the range
    random_values = np.random.uniform(min_value, max_value, size=data[column].isnull().sum())
    
    # Create a Series with the Random Values
    random_series = pd.Series(random_values, index=data[column][data[column].isnull()].index)
    
    # Fill NaN values with the random Series
    data[column].fillna(random_series, inplace=True)
    
    # Print
    # print(data[column].isnull().sum())
    
# data.isnull().sum()

C:\Users\ACER\AppData\Local\Temp\ipykernel_4780\3241618202.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(random_series, inplace=True)


In [73]:
# Separate Features and Target
X = data.drop('Pass/Fail', axis=1)
y = data['Pass/Fail']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)


# Initialize and Train Logistic regression Model
log_reg = LogisticRegression(max_iter=10000) # Increase max_iter if it does not converge
log_reg.fit(X_train, y_train)

# Make prediction on the test set
y_pred = log_reg.predict(X_test)

# Calculate and Print Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

(1253, 590)
(314, 590)
Test Accuracy: 0.9044585987261147


C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


To perform filter-based feature selection on the "UCI SECOM" dataset, which has 592 columns and a target column called "Pass/Fail," we can utilize the following methods:

1. Duplicate Features:
   - Identify and remove duplicate columns from the dataset. Columns with identical values provide redundant information and do not contribute to the prediction task.

2. Variance Threshold Method:
   - Calculate the variance of each feature.
   - Remove features with low variance, as they tend to have little or no predictive power.
   - Set a threshold value for variance and remove features below that threshold.

3. Correlation:
   - Compute the correlation matrix of the features.
   - Identify highly correlated features and choose one from each highly correlated group.
   - High correlation between features indicates redundancy, and removing one from each correlated group helps reduce multicollinearity.

4. ANOVA (Analysis of Variance):
   - Perform an ANOVA test between each feature and the target variable ("Pass/Fail").
   - Select features with a significant impact on the target variable.
   - Set a significance level (e.g., p-value threshold) for the test to determine the importance of each feature.

5. Chi-Squared:
   - Apply the Chi-Squared test between each feature and the target variable, considering both features as categorical.
   - Select features with a significant association with the target variable.
   - Set a significance level (e.g., p-value threshold) to determine the importance of each feature.

Implementing these feature selection methods in Python using the "UCI SECOM" dataset can be done as follows:

In [74]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, f_classif
from scipy.stats import pearsonr

data_path = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQtBXo5cBnDsM2fmfHPm6u72KGUS5FjPHNGMxOfYjA9-CAhmnRpwkIw_rOR3sANJIToiUU__6fbBvig/pub?gid=572763137&single=true&output=csv"

# Load the DataSet
DATA = pd.read_csv(data_path)

# Remove Duplicate Features
# Get the Subset of columns with Duplicate Values
duplicated_cols = DATA.columns[DATA.T.duplicated()]

# Remove the Duplicated Columns
data = DATA.drop(columns=duplicated_cols)

# Drop Time Column
data.drop('Time', inplace=True, axis=1)

# Number of Columns after removing Duplicate Columns
print("Number of Columns - ", DATA.shape[1])
print("Number of Columns after removing duplicate columns- ", data.shape[1])

Number of Columns -  592
Number of Columns after removing duplicate columns-  487


In [75]:
# Variance Threshold Method
selector = VarianceThreshold(threshold=0.01)
sel = selector.fit(data)

columns = data.columns[sel.get_support()]

data_vt = sel.transform(data)

data_vt = pd.DataFrame(data_vt, columns=columns)

# Number of Columns after Variance Threshold Method
print("Number of Columns after Variance Threshold Method- ", data_vt.shape[1])

Number of Columns after Variance Threshold Method-  315


In [76]:
# Correlation

corr_matrix = data_vt.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]

data_corr = data_vt.drop(to_drop, axis=1)

# Number of Columns after Variance Threshold Method
print("Number of Columns after Correlation- ", data_corr.shape[1])

Number of Columns after Correlation-  162


In [77]:
data_corr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 162 entries, 0 to Pass/Fail
dtypes: float64(162)
memory usage: 1.9 MB


ANOVA approach can not be done here because there are NaN values inside this dataset. But we can still use ANOVA to each column individually, comparing the target variable which is Pass/Fail column in our case against the non-missing values in that specific column.

We will drop NaN values of each column before performing the ANOVA

In [78]:
from scipy.stats import f_oneway

# Significance Value
alpha = 0.05

# Columns having p_value less than Alpha
column_pvalues = []

# Iterate over each column
for column in data_corr.iloc[:, :-1].columns:
    # Extract the non-missing values in the column
    column_data = data_corr[column].dropna()
    
    # Perform ANOVA with the Target Variable
    anova_result = f_oneway(column_data, data_corr['Pass/Fail'])
    
    # Print the ANOVA results or perform further analysis
    print(f"Column: {column} - ANOVA p-value: {anova_result.pvalue}")
    
    if anova_result.pvalue <= alpha:
        column_pvalues.append((column, anova_result.pvalue))

# Selecting best 100 Features - Lower P-Value means better feature
# Sort the Column P-Value in Ascending Order
column_pvalues.sort(key=lambda x: x[1])

# Select the Top 100 Columns with the Lowest P-Values
selected_columns = [column for column, _ in column_pvalues[:100]]

data_anova = data_corr[selected_columns+['Pass/Fail']]

print("Number of Columns after Correlation- ", data_anova.shape[1])

Column: 0 - ANOVA p-value: 0.0
Column: 1 - ANOVA p-value: 0.0
Column: 2 - ANOVA p-value: 0.0
Column: 3 - ANOVA p-value: 0.0
Column: 4 - ANOVA p-value: 0.0003805229753154665
Column: 6 - ANOVA p-value: 0.0
Column: 12 - ANOVA p-value: 0.0
Column: 14 - ANOVA p-value: 0.0
Column: 15 - ANOVA p-value: 0.0
Column: 16 - ANOVA p-value: 0.0
Column: 18 - ANOVA p-value: 0.0
Column: 19 - ANOVA p-value: 0.0
Column: 21 - ANOVA p-value: 0.0
Column: 22 - ANOVA p-value: 0.0
Column: 23 - ANOVA p-value: 0.0
Column: 24 - ANOVA p-value: 5.0211194207923185e-05
Column: 25 - ANOVA p-value: 0.0
Column: 28 - ANOVA p-value: 0.0
Column: 29 - ANOVA p-value: 0.0
Column: 31 - ANOVA p-value: 0.0
Column: 32 - ANOVA p-value: 0.0
Column: 33 - ANOVA p-value: 0.0
Column: 34 - ANOVA p-value: 0.0
Column: 35 - ANOVA p-value: 0.0
Column: 37 - ANOVA p-value: 0.0
Column: 38 - ANOVA p-value: 0.0
Column: 39 - ANOVA p-value: 0.0
Column: 40 - ANOVA p-value: 0.0
Column: 41 - ANOVA p-value: 0.0
Column: 43 - ANOVA p-value: 0.0
Column: 4

Chi-Squared cant be done because our Data is Numerical.

In [79]:
# Shape of Data after Feature Selection
print("Shape - ", data_anova.shape)

# Filling NaN values with Random
for column in data_anova.columns:
    
    # Get the minimum and maximum values of the colum
    min_value = data_anova[column].min()
    max_value = data_anova[column].max()
    
    # Generate Random numbers within the range
    random_values = np.random.uniform(min_value, max_value, size=data_anova[column].isnull().sum())
    
    # Create a Series with the Random Values
    random_series = pd.Series(random_values, index=data_anova[column][data_anova[column].isnull()].index)
    
    # Fill NaN values with the random Series
    data_anova[column].fillna(random_series, inplace=True)
    
    # Print
    # print(data_anova[column].isnull().sum())

# Seperate features and target
X = data_anova.drop("Pass/Fail", axis=1)
y = data_anova["Pass/Fail"]
    
# Split Data into Training and Testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
print(X_train.shape)
print(X_test.shape)
    
    
# Initialize and Train Logistic regression Model
log_reg = LogisticRegression(max_iter=10000) # Increase max_iter if it does not converge
log_reg.fit(X_train, y_train)

# Make prediction on the test set
y_pred = log_reg.predict(X_test)

# Calculate and Print Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Shape -  (1567, 101)


C:\Users\ACER\AppData\Local\Temp\ipykernel_4780\3246670434.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_anova[column].fillna(random_series, inplace=True)
C:\Users\ACER\AppData\Local\Temp\ipykernel_4780\3246670434.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_anova[column].fillna(random_series, inplace=True)


(1253, 100)
(314, 100)
Test Accuracy: 0.9203821656050956


C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
